In [26]:
from dotenv import  load_dotenv
load_dotenv()

True

In [14]:
from typing import Dict, TypedDict, Optional


class GraphState(TypedDict):
    question: Optional[str] = None
    classification: Optional[str] = None
    response: Optional[str] = None

In [15]:
def classify(input_string: str) -> str:
    if '?' in input_string:
        return "question"
    return "greeting"

In [27]:
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro")

c:\Users\Coditas\Documents\langgraph\langgraphvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
def classify_input_node(state):
    question = state.get('question', '').strip()
    classification = classify(question)  # Assume a function that classifies the input
    return {"classification": classification}

def handle_greeting_node(state):
    return {"response": "Hello! How can I help you today?"}

def handle_search_node(state):
    question = state.get('question', '').strip()
    search_result = llm.invoke(question)
    return {"response": search_result}

In [29]:
from langgraph.graph import StateGraph
workflow = StateGraph(GraphState)

workflow.add_node("classify_input", classify_input_node)
workflow.add_node("handle_greeting", handle_greeting_node)
workflow.add_node("handle_search", handle_search_node)

def decide_next_node(state):
    return "handle_greeting" if state.get('classification') == "greeting" else "handle_search"

workflow.add_conditional_edges(
    "classify_input",
    decide_next_node,
    {
        "handle_greeting": "handle_greeting",
        "handle_search": "handle_search"
    }
)


In [30]:
from langgraph.graph import END

workflow.set_entry_point("classify_input")
workflow.add_edge('handle_greeting', END)
workflow.add_edge('handle_search', END)

In [31]:
app = workflow.compile()

In [32]:
inputs = {"question": "Hello, how are you"}
result = app.invoke(inputs)
print(result)

{'question': 'Hello, how are you', 'classification': 'greeting', 'response': 'Hello! How can I help you today?'}


In [33]:
result["response"]

'Hello! How can I help you today?'

In [34]:
inputs = {"question": "Capital of India?"}
result = app.invoke(inputs)
print(result)

{'question': 'Capital of India?', 'classification': 'question', 'response': 'New Delhi'}


In [35]:
result["response"]

'New Delhi'